Implement a model to mask various categories with Semantic Segmentation


**Dataset link -** https://www.kaggle.com/kmader/finding-lungs-in-ct-data

**About the dataset:**
- This dataset is a collection of 2D images with manually segmented lungs.
- Using semantic segmentation we will try to find lesions in CT images of the lungs.

## Load all required libraries

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

#for building and training the U-Net model
from tensorflow.keras.layers import Conv2D, Input, Concatenate, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model

print("Loaded all libraries")

## Data loading and data exploration

In [ ]:
train_fpath='../input/finding-lungs-in-ct-data/2d_images/'
train_masks_fpath='../input/finding-lungs-in-ct-data/2d_masks/'

#test_fpath='../input/3d_images/'
#test_masks_fpath='../input/3d_masks/'
print(os.listdir('../input/finding-lungs-in-ct-data/2d_images/'))

In [ ]:
print("No. of images =",len(os.listdir(train_fpath)))
print("No. of image masks =",len(os.listdir(train_masks_fpath)))

In [ ]:
#1 example let's pick image id 'ID_0066_Z_0141'
def plot_img_and_mask(id):
    img = cv2.imread(train_fpath + id + '.tif')
    img_mask = cv2.imread(train_masks_fpath + id + '.tif')
    plt.figure(figsize=(10,10))
    plt.subplot(1,3,1,title='Actual image')
    plt.imshow(img)
    plt.subplot(1,3,3,title='Image mask')
    plt.imshow(img_mask)
    
plot_img_and_mask('ID_0066_Z_0141')

In [ ]:
plot_img_and_mask('ID_0202_Z_0098')

- Initialize training data

In [ ]:
def initialize_img_data(folder):
    lst=[] 
    for image in os.listdir(folder):
        #load image in grayscale
        img= cv2.imread(folder+"/"+image, cv2.IMREAD_GRAYSCALE)
        #convert to array
        img_array=Image.fromarray(img)
        #resize image
        resize_img = img_array.resize((128 , 128))
        #divide by 255 -> scaling data
        norm_img=np.array(resize_img)/255
        #expand dimensions
        img_array = norm_img.reshape((128,128,1))#np.expand_dims(norm_img,axis=3)
        
        lst.append(img_array)
    return lst
    
X = initialize_img_data(train_fpath)
y = initialize_img_data(train_masks_fpath)


In [ ]:
#quick verification whether all the data is initialized or not
print(len(os.listdir(train_fpath)),len(X),len(y))
print(X[0].shape,y[0].shape)

In [ ]:
#check data once they are loaded
plt.figure(figsize=(10,10))
plt.subplot(1,3,1,title='Actual image')
plt.imshow(X[10],cmap="gray")

plt.subplot(1,3,3,title='Image mask')
plt.imshow(y[10],cmap="gray")

In [ ]:
X = np.array(X)
y = np.array(y)

- Split dataset into train, test sets

In [ ]:
x_train = X[:200]
y_train = y[:200]

x_test = X[200:]
y_test = y[200:]

## Build U-Net model

In [ ]:
# Build U-Net model - Define layers
inputs = Input(shape=(128, 128, 1))

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = Concatenate()([u6, c4])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = Concatenate()([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = Concatenate()([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = Concatenate()([u9, c1]) #removed ,axis=3
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

- Initialize the model

In [ ]:
model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model,to_file='model.png')

- Fit the model

In [ ]:
x_train.shape

In [ ]:
early_stop = EarlyStopping(patience=5)
check_point = ModelCheckpoint('model.hdf5',save_best_only=True)
model.fit(x_train, y_train, epochs=200, callbacks=[early_stop,check_point])

In [ ]:
pred_test = model.predict(x_test, verbose=1)

In [ ]:
#exclude channels axis
x_test = np.squeeze(x_test,axis=-1)
pred_test = np.squeeze(pred_test,axis=-1)

print(x_test.shape, pred_test.shape)

- Check the predicted outputs

Predicted masks on test images

In [ ]:
plt.figure(1 , figsize = (15, 9))
n = 0 
for i in range(4):
    n += 1 
    #r = np.random.randint(0, x_test.shape[0], 1)
    r=[i]
    plt.subplot(2, 2, n)
    plt.subplots_adjust(hspace = 0.5, wspace = 0.5)
    plt.imshow(x_test[r[0]], cmap="gray")
    plt.imshow(pred_test[r[0]], alpha=0.5, cmap="Oranges")
    

plt.suptitle("Lung images and the masks predicted by U-Net model highlighted in RED")
plt.show()

Actual masks on test images

In [ ]:
plt.figure(1 , figsize = (15, 9))
n = 0 
for i in range(4):
    n += 1 
    #r = np.random.randint(0, x_test.shape[0], 1)
    r=[i]
    plt.subplot(2, 2, n)
    plt.subplots_adjust(hspace = 0.5, wspace = 0.5)
    plt.imshow(x_test[r[0]], cmap="gray")
    plt.imshow(y_test[r[0]], alpha=0.5, cmap="Oranges")
    

plt.suptitle("Lung images and the masks predicted by U-Net model highlighted in RED")
plt.show()